In [ ]:
!pip install transformers accelerate huggingface-hub

In [ ]:
from huggingface_hub import login
login(token='')  # Replace with token

In [ ]:
import transformers
import torch
import pandas as pd

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Sampled_Posts_1000.csv')

In [ ]:
dataframe=dataframe.astype(str)
dataframe = dataframe.replace('\n','', regex=True)

In [ ]:
dataframe

,postID,post,word_count,category
0,xapb9j,I forgot to take my pill yesterday. Is it okay...,51,Group 1
1,va1s9b,how do i lose weight?. i’ve gained a lot of we...,84,Group 1
2,vyv2ut,Lab results. I just got my labs back and I’d l...,45,Group 1
3,wy5f9k,Can you get rid of the follicles/cysts?. No ot...,61,Group 1
4,yvjy63,hair loss. i’m slowly balding from my scalp. i...,43,Group 1
...,...,...,...,...
995,y87em5,PCOS Beginner Questions. Hi all! I don’t have ...,344,Group 4
996,v96ws6,Insulin Resistance/Sugar Readings. Hello! I wa...,353,Group 4
997,yj6zyi,Weird coincidence??. So I have no idea what’s ...,301,Group 4
998,y6k0n6,Hormone irregularity. I’ll try not to make thi...,346,Group 4


In [ ]:
df_corpus = pd.DataFrame()
df_corpus['postID']=dataframe['postID']
df_corpus['post'] = dataframe['post']

In [ ]:
df_corpus['postID'][0]

'xapb9j'

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
zs_short_base_prompt="""
###Task###
Identify the physical symptoms of Polycystic Ovarian Syndrome (PCOS) mentioned by the narrator in the given post.

###Instructions###
**Include symptoms that are caused due to PCOS (implicit or explicit mention of PCOS).
**Exclude any psychological symptoms.
**List the symptoms separated by commas, or return 'none' if no symptoms identified.
**Do not give any other advice or writing instead the asked answer format.
"""

In [ ]:

def ask_llama(prompt: str):
    messages = [
        {"role": "system", "content": "You are an expert in medical information extraction."},
        {"role": "user", "content": prompt}
    ]

    response = pipeline(
    messages,
    max_new_tokens=250,
    temperature=0.25,
)

    api_response_full= response[0]["generated_text"]

  # Extract the 'content' of the response from the assistant's role
    assistant_response = next(
    (msg["content"] for msg in api_response_full if msg["role"] == "assistant"),
    "No response"
)

    # xtract the response text
    return assistant_response





In [ ]:
responses= []

In [ ]:
for i in range(750,1000):
    postID = df_corpus['postID'][i]
    post = df_corpus['post'][i]
    prompt = zs_short_base_prompt + "\n" + "Post:" + post
    api_response = ask_llama(prompt)
    # Append the valid response
    responses.append({"postID":postID, "post":post, "Symptoms Found(Llama)": api_response})
# Convert the list of responses to a pandas DataFrame
df_responses = pd.DataFrame(responses)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [ ]:
df_responses

,postID,post,Symptoms Found(Llama)
0,xapb9j,I forgot to take my pill yesterday. Is it okay...,none
1,va1s9b,how do i lose weight?. i’ve gained a lot of we...,"acne, excess hair growth, weight gain, irregul..."
2,vyv2ut,Lab results. I just got my labs back and I’d l...,none
3,wy5f9k,Can you get rid of the follicles/cysts?. No ot...,Follicles/cysts on ovaries.
4,yvjy63,hair loss. i’m slowly balding from my scalp. i...,hair loss.
...,...,...,...
995,y87em5,PCOS Beginner Questions. Hi all! I don’t have ...,"hair loss on scalp, weight gain, acne along ja..."
996,v96ws6,Insulin Resistance/Sugar Readings. Hello! I wa...,"Weight gain, nausea, lack of appetite, consist..."
997,yj6zyi,Weird coincidence??. So I have no idea what’s ...,"excess hair growth, acne, irregular periods, v..."
998,y6k0n6,Hormone irregularity. I’ll try not to make thi...,"hirsutism, acne, hot flashes, low energy, weig..."


In [ ]:
df_responses['Symptoms Found(Llama)'][999]

'abnormal bleeding, painful periods, hair thinning, dryness during sex.'

In [ ]:
df_responses.to_excel('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Dataset (Baseline Comparisons)/PCOS Symptoms (Llama 3.1).xlsx', index=False)